# Scrape and parse ria news

There were interruptions - got to run several times.

In [1]:
%%capture
!pip install selenium

In [2]:
import re
import time
import datetime
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

In [38]:
DEPTH = 400
BASE_URL = 'https://ria.ru/'
TOPICS = ['politics', 'world', 
          'economy', 
          'society', 
          'incidents', 'science', 
          'culture', 'religion',
          'defense_safety']

In [15]:
@dataclass
class Article:
    id: str = None
    url: str = None
    title: str = None
    subtitle: str = None
    content: str = None
    datetime: str = None
    views: int = None
    tags: str = None

In [39]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [40]:
import logging
logging.basicConfig(filename='parse_log.txt', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')
    

def get_pages():
    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(1)

            # push to list 20 next articles
            driver.execute_script("document.getElementsByClassName('list-more')[0].click()")
            time.sleep(1)

            # scroll page to automatically load more articles
            for i in tqdm(range(DEPTH), leave=False):
                try:
                    driver.execute_script(f'window.scrollTo(0, document.body.scrollHeight - 1200)')
                    time.sleep(1)
                except Exception as scroll_error:
                    logging.error(f"Error scrolling page: {scroll_error}")

            # find all pages
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            scope = soup.find(
                "div", {"class": "list", "itemtype": "http://schema.org/ItemList"}
            )
            items += scope.find_all("div", {"class": "list-item"})        

            # number of pages can not be multiple of deepth*20
            # that's why we count topics_order dynamically
            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))
                
        except Exception as topic_error:
            logging.error(f"Error processing topic {topic}: {topic_error}")

    return items, topics_order

In [41]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article url
    article.url = page.find('a', {'class': 'list-item__image'})['href']

    # article id
    article.id = re.search(r'\d+.html', article.url).group().split('.')[0]

    # load page
    driver.get(article.url)
    #time.sleep(1)
    html = driver.page_source

    # article source
    source = article.url[8: article.url.find('.')]

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find('div', {'class': lambda x: x and (x.find(f'article m-article m-{source}') > -1),
                                'data-article-id': article.id})

    if not obj:
        obj = soup.find('div', {'class': lambda x: x and (x.find(f'article m-video m-{source}') > -1),
                                'data-article-id': article.id})

    # process article title
    title = obj.find('div', {'class': 'article__title'})
    title_2 = obj.find('h1', {'class': 'article__title'})

    if title:
        article.title = title.text
    else:
        article.title = title_2.text if title_2 else ''

    # article subtitle
    subtitle = obj.find('h1', {'class': 'article__second-title'})
    article.subtitle = subtitle.text if subtitle else ''

    # article content
    article_body = obj.find('div', {'class': 'article__body js-mediator-article mia-analytics'})
    full_text = ""
    if article_body and article_body.find_all():
            content = article_body.find_all('div', {'data-type': ['text', 'quote']})
            full_text = " ".join([block.text for block in content])
    #Тексты анонсов  
    elif obj.find('div', {'class': 'article__header'}):
        full_text = obj.find('div', {'class': 'article__header'}).find('div', {'class': 'article__announce-text'}).text
                
    article.content  = full_text.strip()

    # article datetime
    article.datetime = obj.find('div', {'class': 'article__info-date'}).find('a').text

    # article number of views 
    numb_views = obj.find('span', class_='article__views')
    article.views = int(re.findall(r'\d+', numb_views.get_text(strip=True))[0]) if numb_views else 0
                   
    #article tags
    article.tags = [topic.get_text(strip=True) for topic in obj.find_all('a', class_='article__tags-item')]

    return article

In [42]:
# get pages and topics
pages, topics_order = get_pages()

100%|██████████| 3/3 [06:04<00:00, 121.39s/it]   


In [35]:
len(pages)

948

In [36]:
len(topics_order)

948

In [43]:
# parse each page and get desired attributes
# few pages are differ significantly from others
# using try/except we can ignore them

data, topics_order_fixed = [], []

for num, page in enumerate(tqdm(pages)):
    try:
        res = parse_page(page)
        data.append(res)
        topics_order_fixed.append(topics_order[num])

        if num % 10 == 0:
            df = pd.DataFrame(data)
            df['topic'] = topics_order_fixed
            df.to_csv('ria_news_topics_10.csv', index=False)
            
    except Exception as e:
        logging.error(f"Error processing page {num}: {e}")
        continue

driver.close()

100%|██████████| 4436/4436 [5:31:49<00:00,  4.49s/it]   
